## Design Patterns with First-Class Functions

### Class strategy

In [20]:
from abc import ABC, abstractclassmethod
from collections import namedtuple

In [21]:
Customer = namedtuple("Customer", "name fidelity")

In [22]:
class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price 
        
    def total(self):
        return self.price * self.quantity
    
    def __repr__(self):
        return "item:{}, qty:{}, price:{} total price:{}".format(self.product, self.quantity, self.price, self.quantity)

In [23]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, "__total"):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = "<Order total: {:.2f} due {:.2f}>"
        return fmt.format(self.total(), self.due())

In [24]:
class Promotion(ABC):    # the strategy: an abstract base class
    
    @abstractclassmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""

In [25]:
class FidelityPromo(Promotion):  #First concrete strategy
    """5% discount for customers with 1000 of more fidelity points"""
    
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [26]:
class BulkItemPromo(Promotion): #second concrete strategy 
    """10% discount for each LineItem with 20 or more units"""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

In [ ]:
class LargeOrderPromo(Promotion):  #third concrete strategy
    """7% discount for orders with 10 or more distinct items"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() *.07
        return 0

In [28]:
joe = Customer("John Doe", 0)
ann = Customer("Ann Smith", 1100)

In [29]:
cart = [LineItem("banana",4, .5),
        LineItem("apple", 10, 1.5),
        LineItem("watermelon", 5, 5.0)]

In [30]:
cart

[item:banana, qty:4, price:0.5 total price:4,
 item:apple, qty:10, price:1.5 total price:10,
 item:watermelon, qty:5, price:5.0 total price:5]

In [31]:
banana_cart = [LineItem("banana", 30, .5),
               LineItem("apple", 10, 1.5)]

In [32]:
banana_cart

[item:banana, qty:30, price:0.5 total price:30,
 item:apple, qty:10, price:1.5 total price:10]

In [33]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
long_order

[item:0, qty:1, price:1.0 total price:1,
 item:1, qty:1, price:1.0 total price:1,
 item:2, qty:1, price:1.0 total price:1,
 item:3, qty:1, price:1.0 total price:1,
 item:4, qty:1, price:1.0 total price:1,
 item:5, qty:1, price:1.0 total price:1,
 item:6, qty:1, price:1.0 total price:1,
 item:7, qty:1, price:1.0 total price:1,
 item:8, qty:1, price:1.0 total price:1,
 item:9, qty:1, price:1.0 total price:1]

In [16]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due 42.00>

In [17]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due 39.90>

In [34]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due 28.50>

In [35]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due 3.00>

In [36]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due 42.00>

&nbsp;

## Function oriented strategy

In [43]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [44]:
from collections import namedtuple

Customer = namedtuple("Customer", "name fidelity")

In [51]:
class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity 
        self.price = price 
        
    def total(self):
        return self.price * self.quantity
    
    def __repr__(self):
        return "item:{}, qty:{}, price:{} total price:{}".format(self.product, self.quantity, self.price, self.quantity)

In [ ]:
class Order:
    
    def __init__(self, customer, cart, promotion):
        self.customer = customer 
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, "__total"):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = "<Order total: {:.2f} due: {:.2f}>"
        return fmt.format(self.total(), self.due())
    

In [63]:
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order): #second concrete strategy 
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):  #third concrete strategy
    """7% discount for orders with 10 or more distinct items"""

    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() *.07
    return 0

In [53]:
joe = Customer("John Doe", 0)
ann = Customer("Ann Smith", 1100)

In [54]:
cart = [LineItem("banana",4, .5),
        LineItem("apple", 10, 1.5),
        LineItem("watermelon", 5, 5.0)]

In [55]:
cart

[item:banana, qty:4, price:0.5 total price:4,
 item:apple, qty:10, price:1.5 total price:10,
 item:watermelon, qty:5, price:5.0 total price:5]

In [56]:
banana_cart = [LineItem("banana", 30, .5),
               LineItem("apple", 10, 1.5)]

In [57]:
banana_cart

[item:banana, qty:30, price:0.5 total price:30,
 item:apple, qty:10, price:1.5 total price:10]

In [58]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
long_order

[item:0, qty:1, price:1.0 total price:1,
 item:1, qty:1, price:1.0 total price:1,
 item:2, qty:1, price:1.0 total price:1,
 item:3, qty:1, price:1.0 total price:1,
 item:4, qty:1, price:1.0 total price:1,
 item:5, qty:1, price:1.0 total price:1,
 item:6, qty:1, price:1.0 total price:1,
 item:7, qty:1, price:1.0 total price:1,
 item:8, qty:1, price:1.0 total price:1,
 item:9, qty:1, price:1.0 total price:1]

In [61]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [62]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [65]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
long_order

[item:0, qty:1, price:1.0 total price:1,
 item:1, qty:1, price:1.0 total price:1,
 item:2, qty:1, price:1.0 total price:1,
 item:3, qty:1, price:1.0 total price:1,
 item:4, qty:1, price:1.0 total price:1,
 item:5, qty:1, price:1.0 total price:1,
 item:6, qty:1, price:1.0 total price:1,
 item:7, qty:1, price:1.0 total price:1,
 item:8, qty:1, price:1.0 total price:1,
 item:9, qty:1, price:1.0 total price:1]

In [67]:
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [68]:
Order(ann, cart, bulk_item_promo)

<Order total: 42.00 due: 42.00>